# WHERE SHOULD I OPEN A RESTAURANT?

I AM PLANNING TO OPEN A RESTAURANT IN SAN JOSE. I AM LOOKING FOR THE RIGHT CUISINE AND THE RIGHT LOCATION SO THAT I CAN GAIN HUGE PROFITS, BUT HOW CAN I CHOOSE?

In [4]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import requests
from geopy.geocoders import Nominatim
import json
from pandas import json_normalize
from sklearn.cluster import KMeans

In [5]:
client_id = "P1BRDUNBKGVA44LZNLSJINAVHF52WT5IBWFUM5YNTYKJR2V1"
client_secret = "5QIX2Y4O2EHPH51USGIS0F32VLB0EGH4QZTYQKCVVPWOQS2J"

In [6]:
#address coordinates
address = "San Jose,CA"

locator = Nominatim(user_agent = "foursquare_api")
coordinates = locator.geocode(address)
latitude = coordinates.latitude
longitude = coordinates.longitude

GeocoderTimedOut: Service timed out

In [ ]:
latitude , longitude

In [ ]:
sj_map = folium.Map(location = [latitude,longitude],zoom_start = 12)

In [ ]:
sj_map

In [ ]:
#fetching the json file the restaurants in new york

In [ ]:
#first let us make a list of the top cuisines in bangalore
cuisines = ['American','Italian','French','Indian',
            'Chinese','Japanese','Thai','Mexican',
           'Spanish']
cat_id = ['4bf58dd8d48988d14e941735','4bf58dd8d48988d110941735','4bf58dd8d48988d10c941735','4bf58dd8d48988d10f941735',
          '4bf58dd8d48988d145941735','4bf58dd8d48988d111941735','4bf58dd8d48988d149941735','4bf58dd8d48988d1c1941735',
         '4bf58dd8d48988d150941735']

In [ ]:
df = pd.DataFrame({})
for index,code in enumerate(cat_id):
    url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}".format(client_id, client_secret, latitude, longitude,20200404,code, 6000,100)
    results = requests.get(url).json()
    venues = results['response']['venues']
    data = json_normalize(venues)
    data["cuisine"] = cuisines[index]
    df = pd.concat([df,data],axis=0)
    print(index,cuisines[index])

In [ ]:
df

In [ ]:
len(df["id"].unique())

In [ ]:
nan = [np.nan]*df.shape[0]
df["rating"] = nan
df.head()

In [ ]:
for i in range(df.shape[0]):
    venue_id = df.iloc[i,0]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id,client_id, client_secret,20200404)
    result = requests.get(url).json()
    try:
        df.iloc[i,-1] = result['response']['venue']['rating']
    except:
        continue
    

In [ ]:
color_code = {'American':'red','Italian':'yellow','French':'green','Indian':'blue',
            'Chinese':'pink','Japanese':'brown','Thai':'purple','Mexican':'orange','Spanish':'black'}

for lat, lng, name,cuisine in zip(df["location.lat"], df["location.lng"], df["name"], df["cuisine"]):
    label = '{}, {}'.format(name, cuisine)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color_code[cuisine],
        fill=True,
        fill_color= color_code[cuisine],
        fill_opacity=0.7,
        parse_html=False).add_to(sj_map)  
    
sj_map

In [ ]:
# we dont have any information about the ratings of the restaurant, 
# so we will use the count as a parameter to see the top rated cuisine of San Jose
# we also have a limit to the number of results per call
# we will take cuisine with the highest count to be the cuisine with the

In [ ]:
df["cuisine"].value_counts()

In [ ]:
# we can see that french and spanish cuisines are the lowest in number , so investing in these can be a good idea
# be need to choose a location cluster where the number of restaurants are high but the number of spanish or french
# restaurants are low

In [ ]:
# lets cluster the neighborhood. 
# first let us choose the correct value of k

In [ ]:
k = [1,2,3,4,5,6,7,8,9,10]
inertia = []
for i in k:
    kmeans = KMeans(n_clusters = i)
    kmeans.fit(df[["location.lat","location.lng"]])
    inertia.append(kmeans.inertia_)

In [ ]:
plt.plot(k,inertia)

In [ ]:
# from the elbow method we can see that 5 clusters seem to be the appropriate
kmeans = KMeans(n_clusters = 5)
kmeans.fit(df[["location.lat","location.lng"]])

In [ ]:
cluster = kmeans.labels_

In [ ]:
map_clusters = folium.Map(location = [latitude,longitude],zoom_start = 12)

rainbow = ["red","blue","orange","yellow","purple"]

# add markers to the map
markers_colors = []
for lat, lon, name,cuisine, cluster in zip(df['location.lat'], df['location.lng'], df['name'],df['cuisine'], kmeans.labels_):
    label = folium.Popup(str(name)+ " " + str(cuisine), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# red - cluster 0
# blue - cluster 1
# orange - cluster 2
# yellow - cluster 3
# purple - cluster 4

In [ ]:
df["cluster"] = kmeans.labels_

In [ ]:
df.head()

In [ ]:
df["cluster"].value_counts()

In [ ]:
kmeans.labels_

In [ ]:
for i in range(5):
    print("Cluster ",i)
    print(df.loc[df["cluster"] == i]["cuisine"].value_counts())
    print("__________________________________________")

In [ ]:
# we can see that cluster 0 (red) is closer to the city compared to the cluster 1 (blue)
# cluster 4 has the highest number of restaurants but it also has only a few spanish and french restaurant

In [ ]:
# so opening a spanish restaurant in cluster 4 seems good

In [ ]:
# cluster 4 center
lat,long = kmeans.cluster_centers_[4]

In [ ]:
lat,long

In [ ]:
# so a spanish or a french restaurant in the vicinity of coordinates (37.32683590424944, -121.89186990586737)
# would be highly profitable